# Connect to Google Search Console API using Python
This notebook shows you how to make a simple Google Search Console API call with filters using Python.

@author: Jean-Christophe Chouinard: Senior SEO Specialis at Seek.

[LinkedIn](https://www.linkedin.com/in/jeanchristophechouinard), [Twitter.com/@ChouinardJC](https://twitter.com/chouinardjc), Blog > [jcchouinard.com](https://www.jcchouinard.com)

Before you can run this code, make sure that you know how to get your [Google Search Console API Keys](https://www.jcchouinard.com/how-to-get-google-search-console-api-keys/)

In [1]:
import pandas as pd
import datetime
import httplib2
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow
from collections import defaultdict
from dateutil import relativedelta
import argparse
from oauth2client import client
from oauth2client import file
from oauth2client import tools
import re
import os

## Get Your Path and Set-Up Your GSC Domain

In [2]:
path = os.getcwd()
site = 'https://www.example.com'

## Set Credentials
To run this, you need to add your `client_secrets.json` file to your current working directory. If you don't know how, just check how you can [download your Google API keys](https://www.jcchouinard.com/how-to-get-google-search-console-api-keys/).

The first time your run this, you will get an error message, because it will add the `/searchconsolereporting.dat` file to your current working directory.

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly'] # Variable parameter that controls the set of resources that the access token permits.

CLIENT_SECRETS_PATH = path + '/client_secrets.json' # Path to client_secrets.json file.
 
parser = argparse.ArgumentParser(
    formatter_class = argparse.RawDescriptionHelpFormatter,
    parents = [tools.argparser])
flags = parser.parse_args([])

flow = client.flow_from_clientsecrets(
    CLIENT_SECRETS_PATH, scope = SCOPES,
    message = tools.message_if_missing(CLIENT_SECRETS_PATH))

storage = file.Storage(path + '/searchconsolereporting.dat')
credentials = storage.get()


In [ ]:
if credentials is None or credentials.invalid:
  credentials = tools.run_flow(flow, storage, flags)
http = credentials.authorize(http=httplib2.Http())

webmasters_service = build('webmasters', 'v3', http=http)

## Set-Up Parameters

In [ ]:
start_date = datetime.date.today() - relativedelta.relativedelta(days=3)
end_date = start_date

In [ ]:
maxRows = 25000; 
scDict = defaultdict(list)
print(scDict)

## Execute your API Request

In [ ]:
def execute_request(service, property_uri, request):
    return service.searchanalytics().query(siteUrl=property_uri, body=request).execute()

## Run Request

In [ ]:
request = {
        'startDate': datetime.datetime.strftime(start_date,"%Y-%m-%d"),
        'endDate': datetime.datetime.strftime(end_date,'%Y-%m-%d'),
        'dimensions': ['date','page','query'],  #country, device, page, query, searchAppearance
        'dimensionFilterGroups': [{
          'filters': [{
              'dimension': 'page',              
              'operator': 'contains',           #contains, equals, notEquals, notContains
              'expression': '/job/'
          }]
         }],
        'rowLimit': maxRows 
    }
response = execute_request(webmasters_service, site, request)


In [ ]:
for row in response['rows']:
        scDict['date'].append(row['keys'][0] or 0)    
        scDict['page'].append(row['keys'][1] or 0)
        scDict['query'].append(row['keys'][2] or 0)
        scDict['clicks'].append(row['clicks'] or 0)
        scDict['ctr'].append(row['ctr'] or 0)
        scDict['impressions'].append(row['impressions'] or 0)
        scDict['position'].append(row['position'] or 0)

In [ ]:
df = pd.DataFrame(data = scDict)
df['clicks'] = df['clicks'].astype('int')
df['ctr'] = df['ctr']*100
df['impressions'] = df['impressions'].astype('int')
df['position'] = df['position'].round(2)
df.sort_values('clicks',inplace=True,ascending=False)
df